In [ ]:
Fe = eb.get_element(26)
@nb.njit
def artif_spec(ekin, bg, delta_ekin, fwhm, w_he, w_li, w_be, w_b, w_c, w_n, w_o):
    _, xs = eb.xs.drxs_energyscan(Fe, fwhm, ekin + delta_ekin)
    he = w_he * xs[-3]
    li = w_li * xs[-4]
    be = w_be * xs[-5]
    b = w_b * xs[-6]
    c = w_c * xs[-7]
    n = w_n * xs[-8]
    o = w_o * xs[-9]
    return (he + li + be + b + c + n + o) * 1.e24 + bg



In [16]:
from lmfit import Model
artif_model = Model(artif_spec)
artif_model.set_param_hint("bg", min=0)
artif_model.set_param_hint("delta_ekin", min=-250, max=100)
artif_model.set_param_hint("fwhm", min=0.1, max=100)
artif_model.set_param_hint("w_he", min=0, max=1000)
artif_model.set_param_hint("w_li", min=0, max=1000)
artif_model.set_param_hint("w_be", min=0, max=1000)
artif_model.set_param_hint("w_b", min=0, max=1000)
artif_model.set_param_hint("w_c", min=0, max=1000)
artif_model.set_param_hint("w_n", min=0, max=1000)
artif_model.set_param_hint("w_o", min=0, max=1000)
artif_model.make_params()

name,value,initial value,min,max,vary
bg,-inf,None,0.00000000,inf,True
delta_ekin,-inf,None,-250.000000,100.000000,True
fwhm,-inf,None,0.10000000,100.000000,True
w_he,-inf,None,0.00000000,1000.00000,True
w_li,-inf,None,0.00000000,1000.00000,True
w_be,-inf,None,0.00000000,1000.00000,True
w_b,-inf,None,0.00000000,1000.00000,True
w_c,-inf,None,0.00000000,1000.00000,True
w_n,-inf,None,0.00000000,1000.00000,True
w_o,-inf,None,0.00000000,1000.00000,True


In [ ]:
data = hist[:, -10:].mean(axis=-1)
x = .5*ex[1:] + .5*ex[:-1]
y = .5*ey[1:] + .5*ey[:-1]

e_kin = 4400 + 800/(x[-1]-x[0]) * (x-x[0])
t = 7/(y[-1]-y[0]) * (y-y[0])


res = artif_model.fit(
    data, ekin=e_kin,
    bg=100,
    delta_ekin=-90,
    fwhm=3,
    w_he=10,
    w_li=10,
    w_be=10,
    w_b=10,
    w_c=10,
    w_n=10,
    w_o=10,
    calc_covar=True,
#     method="basinhopping"
)

In [ ]:
res

In [ ]:
res.plot(data_kws={"ms":1}, fig_kws={"figsize":(10,10)})

In [ ]:
from tqdm import tqdm
nrows = 10
strips = hist.shape[1]//nrows
popts = []
stdes = []
# del res
for k in tqdm(range(strips)[::-1]):
    
    data = hist[:, k*nrows:(k+1)*nrows].mean(axis=-1)

    try:
        res = artif_model.fit(
            data, ekin=e_kin,
            bg=100,
            delta_ekin=-90,
            fwhm=15,
            w_he=10,
            w_li=10,
            w_be=10,
            w_b=10,
            w_c=10,
            w_n=10,
            w_o=10,
            calc_covar=True,
#             method="bfgs"
        )
        popts.append([p.value for k, p in res.params.items()])
        stdes.append([p.stderr for k, p in res.params.items()])
    except TypeError:
        popts.append(np.zeros(10))
        stdes.append(np.zeros(10))
popts = np.array(popts[::-1])
stdes = np.array(stdes[::-1])
    

In [ ]:
plt.subplots(figsize=(15,15))
plt.plot(popts[:, 3:],".-")
plt.ylim(0,100)

In [ ]:
for i in range(3, popts.shape[1]):
    plt.fill_between(range(strips), popts[:, i] - stdes[:, i], popts[:, i] + stdes[:, i])

In [ ]:
popts

In [ ]:
_ = eb.plot_combined_xs(26,20, legend=False)